<table style="width: 100%;">
    <tr style="background-color: transparent;"><td>
        <img src="https://d8a-88.github.io/econ-fa19/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 12pt;"><strong>Economic Models</strong>, Spring 2020<br>
            Dr. Eric Van Dusen<br>
        Alan Liang<br>
        Umar Maniku<br></p></td></tr>
</table>

<!-- BEGIN QUESTION -->

# Homework 3: Taxes, Government Intervention, and Welfare

<!-- END QUESTION -->

In [ ]:
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
import sympy
solve = lambda x,y: sympy.solve(x-y)[0] if len(sympy.solve(x-y))==1 else "Not Single Solution"
%matplotlib inline
import otter
grader = otter.Notebook()

## Question 1: Beer Taxes: an Empirical Study

Many states in the United States impose an excise and an ad valorem tax on the consumption of beer. According to the Beer Institute, “taxes are the single most expensive ingredient in beer, costing more than the labor and raw materials combined.” In fact, approximately 40 percent of the retail price of beer is dedicated toward covering all the applicable taxes. 

Throughout this exercise, you will investigate how salient consumers are to changes in taxes on beer. Typically, the ad valorem tax on beer is not reflected in the label price, like typical sales taxes in the US. However, the excise tax is included in the label price. 

This study on consumer salience towards taxes is taken from [_Salience and Taxation: Theory and Evidence_](https://www.aeaweb.org/articles?id=10.1257/aer.99.4.1145) by Chetty, Looney, and Kroft (AER 2010). 

<!-- BEGIN QUESTION -->

**Question 1.1:**
In the paper *Salience and Taxation: Theory and Evidence*, the authors also investigate consumer salience towards taxes through a field experiment by posting tax-inclusive price tags, and find that doing so reduces demand by 8 percent. What does this imply about consumer salience?

_Write your answer here, replacing this text_

<!-- END QUESTION -->

Let's load in a table about beer sales across the 50 states from 1970 to 2003.

The table `beer_raw` includes 7 columns:
- `st_name`: the state abbreviation
- `year`: the year the data was recorded
- `c_beer`: the quantity of beer consumed in gallons
- `beer_tax`: the ad valorem tax as a percentage
- `btax_dollars`: the excise tax, represented in dollars per case of beer (24 cans)
- `population`: the population of the state (in thousands)
- `salestax`: the sales tax percentage

In [ ]:
beer_raw = Table.read_table("beer_tax.csv").drop("Unnamed: 0")
beer_raw

**Question 1.2:**
Consumption across states vary widely due to differences in population. Let's normalize this by adding a column called `pc_beer`, representing the per capita (per person) consumption of beer in each year for each state.

In [ ]:
pc_beer_column = ...
beer = ...

In [ ]:
grader.check("q1_2")

**Question 1.3:**
For the rest of this question, we will restrict our analysis to only one state - California. Assign `beer_state` to a filtered table with only data describing California.

In [ ]:
beer_state = ...
beer_state

In [ ]:
grader.check("q1_3")

**Question 1.4:**
To determine consumers' sensitivity to a change in taxes, we need to determine the changes in ad valorem taxes between each year and the corresponding change in the per capita beer consumption. Construct a new table `beer_state_changes` with these  2 columns: `tax_diff` and `consumption_diff`.

_Hint:_ Use `np.diff`.

In [ ]:
tax_diff = ...
per_capita_consumption_diff = ...
beer_state_changes = ...
beer_state_changes

In [ ]:
grader.check("q1_4")

**Question 1.5:**
Now we are ready to plot our results. What kind of graph will be most appropriate to denote the relationship between changes in taxes and changes in consumption? Assign `q5` to your choice.

1. Line plot
2. Histogram
3. Scatter plot
4. Bar chart

In [ ]:
q1_5 = ...

In [ ]:
grader.check("q1_5")

<!-- BEGIN QUESTION -->

**Question 1.6:**
Plot the results from `beer_state_changes`. What is the relationship between changes in taxes and changes in per capita beer consumption?

In [ ]:
...

_Write your answer here, replacing this text_

<!-- END QUESTION -->

We will conduct a linear regression to determine the slope and intercept.

In [ ]:
beer_state_changes.scatter(0,1, fit_line=True)

What are the values of the intercept and slope terms? 

To answer this, we can use `np.polyfit` with degree 1, which returns the slope and intercept terms in that order.

In [ ]:
np.polyfit(x = beer_state_changes.column("tax_diff"), y=beer_state_changes.column("consumption_diff"), deg=1)

<!-- BEGIN QUESTION -->

**Question 1.7:**
Interpret the slope and intercept values in the previous part in context of this dataset.

_Write your answer here, replacing this text_

**Question 1.8:**
Now let's see if the phenomenon applies to all states, across all years.

<!-- END QUESTION -->

In [ ]:
# You do not have to understand how the code below works, but it works very similarly to what you've done above!
import pandas as pd
data = pd.read_stata("alcohol_data.dta")
data = data[["st_name", "year", "c_beer", "population", "beer_tax", "salestax"]]
data['beer_pc'] = data.c_beer / data.population
data = data.dropna(axis = 0)
def state_group(df):
    tax_diff = np.diff(df["beer_tax"])
    cons_diff = np.diff(df["beer_pc"])
    return pd.DataFrame({"tax_diff": tax_diff, "cons_diff": cons_diff})
differences = data.groupby("st_name").apply(state_group)
diff = Table.from_df(differences)
diff.scatter(0, 1)

The data seem to be a little messy, but a downward sloping trend seems to appear. Nonetheless, let's try to clean it up a little. 

We will aggregate the data based on state so that each state will be one data point. Each point represents a state's average change in taxes and consumption across the years.

In [ ]:
# You also do not have to understand how the code below works.
def state_group(df):
    tax_diff = np.mean(np.diff(df["beer_tax"]))
    cons_diff = np.mean(np.diff(df["beer_pc"]))
    return pd.DataFrame({"tax_diff": tax_diff, "cons_diff": cons_diff}, index=[0])
differences = data.groupby("st_name").apply(state_group)
diff = Table.from_df(differences)
diff.scatter(0, 1)

<!-- BEGIN QUESTION -->

Which graph better represents the relationship between changes in taxes and beer consumption? Which graph do you prefer? Why?

_Write your answer here, replacing this text_

**Qustion 1.9:**
Up until now, we've been examining the effect of changes in the ad valorem tax on beer consumption. But what about the excise tax? Unlike ad valorem taxes, an excise tax is included as part of the label price.

Intuitively, do you expect consumers be more elastic towards changes in the ad valorem tax or excise tax? Why?

_Write your answer here, replacing this text_

<!-- END QUESTION -->

Let's put this theory to the test. Each row in the table `excise_changes` displays an event in which there was a change in the excise tax, and its corresponding change in beer consumption. For example, from the table we can see that Arkansas has historically changed its beer excise tax three times. 

In [ ]:
# You do not need to understand how this code works.
excise_changes = pd.DataFrame({"st_name":[], "excise_tax_change":[], "c_beer_change":[]})
beer_df = beer.to_df()
for row_index in range(beer_df.shape[0]):
    if row_index == 0:
        continue
    if beer_df.iloc[row_index, 4] != beer_df.iloc[row_index-1, 4] and beer_df.iloc[row_index, 0] == beer_df.iloc[row_index-1, 0]:
        excise_changes = excise_changes.append({"st_name": beer_df.iloc[row_index, 0], 
                                                "excise_tax_change": beer_df.iloc[row_index, 4] - beer_df.iloc[row_index-1, 4],
                                                "c_beer_change": beer_df.iloc[row_index, 7] - beer_df.iloc[row_index-1, 7]}, ignore_index=True)
excise_changes = Table.from_df(excise_changes[["st_name", "excise_tax_change", "c_beer_change"]])
excise_changes

In [ ]:
excise_changes.scatter(1,2,fit_line=True)

<!-- END QUESTION -->

**Question 1.10:**
Using `np.polyfit`, determine the slope and intercept terms for this regression. Then, interpret the results.

*Hint:* Be aware of the units of the excise tax!

In [ ]:
coeffs = np.polyfit(x = ..., y = ..., deg = ...)
coeffs

_Write your answer here, replacing this text_


In [ ]:
grader.check("q1_10")

<!-- BEGIN QUESTION -->

**Question 1.11:**
We are unable to do a direct comparison between the slopes because of the different units involved for the slope. As a result, we will have to adjust the units. 

Currently, a case of beer costs around 24 dollars, which means that a 1 dollar change in the excise tax corresponds to a $\frac{1}{24}=4.2$ percent change. Convert the slope due to a dollar change in the excise tax to be in reference to a percent change in the excise tax, based on the assumption that a case of beer is 24 dollars. Compare this value to the slope due to the ad valorem tax.



_Write your answer here, replacing this text_

**Question 1.12:**
Is this result in line with what you had expected in question 2.9? Why or why not? What could be some potential sources of error?

_Write your answer here, replacing this text_


<!-- END QUESTION -->

## Question 2: Tax Equilibria with SymPy

*From Gruber, Ch.19, Q.4*

The demand for football tickets is $Q = 360 − 10P$ and the supply of football tickets is $Q = 20P$. 

**Question 2.1:**
Calculate the quantity and price at the equilibrium.

In [ ]:
P = sympy.Symbol("P")
supply = ...
demand = ...
price_no_tax = ...
quantity_no_tax = ...
print("The equilibrium price is", price_no_tax, "; the equilibrium quantity is", quantity_no_tax)

In [ ]:
grader.check("q2_1")

**Question 2.2:**
Assume that the government implements a per-ticket tax of $4 when consumers purchase a ticket. What is the new price received by producers and price paid by consumers?

In [ ]:
demand_with_tax = ...
price_received_with_tax = ...
price_paid_demand_with_tax = ...
print("The price received by producers is", price_received_with_tax, 
      "; the price paid by consumers is", price_paid_demand_with_tax)

In [ ]:
grader.check("q2_2")

<!-- BEGIN QUESTION -->

**Question 2.3:**
Who bears a larger burden of the tax? Why?

_Write your answer here, replacing this text_

<!-- END QUESTION -->

**Question 2.4:**
Continue to assume that the government implements a per-ticket tax of \\$4. How many less tickets are sold due to the tax?

In [ ]:
quantity_with_tax = ...
difference_in_quantity = ...
print("The difference in quantity transacted is", difference_in_quantity)

In [ ]:
grader.check("q2_4")

<!-- END QUESTION -->

---

Congrats on finishing a notebook! To double-check your work, the cell below will rerun all of the autograder tests for you.

In [ ]:
grader.check_all()

## Submission

Congrats on finishing a homework notebook! To turn in this homework assignment, click the **Save** icon at the top, **save this file** by going to File > Download As and select **Notebook**; then, run the cell below to generate a PDF of this assignment and download it. Submit this assignment by uploading **BOTH the .ipynb and .pdf files** to Gradescope.

In [ ]:
grader.export("hw03.ipynb")